In [51]:
import cv2
# If you want to read a video file set the path under path, 
# If you want to use a specific image set use_video to False and give the image path under path
#TODO: Set the path to the video file or image file
use_video = True
path = ""

#TODO: Set the task name here -- this will be used to save the output
task_name = "curr_task"

img = None
if use_video:
    cap = cv2.VideoCapture(path)
    while(cap.isOpened()):
        ret, frame = cap.read()
        img = frame.copy()
        break
else:
    img = cv2.imread(path)

# We flip here because CV2 reads in as BGR
img = img[:,:,::-1]

#TODO: If its hard to see the image, you can increase the size_multiplier, this won't affect the selected coordinates
size_multiplier = 1

coordinates_path = f"../../coordinates"

[ERROR:0] global /tmp/pip-req-build-3umofm98/opencv/modules/videoio/src/cap.cpp (160) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.3) /tmp/pip-req-build-3umofm98/opencv/modules/videoio/src/cap_images.cpp:293: error: (-215:Assertion failed) !_filename.empty() in function 'open'




TypeError: 'NoneType' object is not subscriptable

In [49]:
%gui asyncio

import os
from PIL import Image

import numpy as np
from IPython.display import display, Javascript
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
import pickle

import io
import asyncio
import logging

# Define an async function to wait for button click
async def wait_for_click(button):
    # Create a future object
    future = asyncio.Future()
    # Define the click event handler
    def on_button_clicked(b):
        future.set_result(None)
    # Attach the event handler to the button
    button.on_click(on_button_clicked)
    # Wait until the future is set
    await future

class Points():
    def __init__(self, env_name, img, coordinates_path, size_multiplier=1):
        logging.getLogger().setLevel(logging.DEBUG)
        logging.info("Starting the Points class")
        self.img = img
        self.size_multiplier = size_multiplier
        self.coordinates_path = coordinates_path
        self.env_name = env_name

        # Save the image to a bytes buffer
        image = Image.fromarray(self.img)
        size = img.shape
        image = image.resize((size[1] * self.size_multiplier, size[0] * self.size_multiplier))
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        buffer.seek(0)

        # Create an IPyWidgets Image widget
        self.canvas = Canvas(width=size[1] * self.size_multiplier, height=size[0] * self.size_multiplier)
        # Define the size of each cell

        self.canvas.put_image_data(np.array(image), 0, 0)

        # Display coordinates
        coords_label = widgets.Label(value="Click on the image to select the coordinates")

        # Define the click event handler
        self.coords = []
        def on_click(x, y):
            coords_label.value = f"Coordinates: ({x}, {y})"
            self.coords.append((0, x, y))

            with hold_canvas(self.canvas):
                self.canvas.put_image_data(np.array(image), 0, 0)  # Redraw the original image

                self.canvas.fill_style = 'red'
                for coord in self.coords:
                    x, y = coord[1] // self.size_multiplier, coord[2] // self.size_multiplier
                    self.canvas.fill_circle(x, y, 5)

        # Connect the click event to the handler
        self.canvas.on_mouse_down(on_click)

        self.button = widgets.Button(description="Save Images")

        # Display the widgets
        self.vbox = widgets.VBox([self.canvas, coords_label, self.button])

        # # Display the widget
        display(self.vbox)

    def on_done(self):
        logging.info("saving")
        with open(self.coordinates_path + "/coords/" + self.env_name + ".pkl", 'wb') as f:
            try:
                pickle.dump(self.coords, f)
            except Exception as e:
                logging.info(e)
        with open(self.coordinates_path + "/images/" + self.env_name + ".png", 'wb') as f:
            try:
                image = Image.fromarray(self.img)
                image.save(f)
            except Exception as e:
                logging.info(e)
        logging.info("saved")

In [50]:
async def f():
    point = Points(task_name, img, coordinates_path, size_multiplier)
    x = await wait_for_click(point.button)
    point.vbox.close()
    point.canvas.close()
    point.on_done()
asyncio.ensure_future(f())

<Task pending name='Task-21' coro=<f() running at /tmp/ipykernel_2605232/3187731821.py:1>>

INFO:root:Starting the Points class


DEBUG:Comm:handle_msg[405f8c52770f434f80048e2e99b03133]({'header': {'date': datetime.datetime(2024, 10, 8, 20, 28, 29, 741000, tzinfo=tzutc()), 'msg_id': 'f04c097d-4dec-4f9e-9fcb-134510e3f7bf', 'msg_type': 'comm_msg', 'session': '28950468-58ca-494e-b47f-fbd55678ef50', 'username': '18153ef9-50e0-4d4b-ad84-32d9963a420e', 'version': '5.2'}, 'msg_id': 'f04c097d-4dec-4f9e-9fcb-134510e3f7bf', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': '405f8c52770f434f80048e2e99b03133', 'data': {'method': 'custom', 'content': {'event': 'client_ready'}}}, 'buffers': []})
DEBUG:Comm:handle_msg[405f8c52770f434f80048e2e99b03133]({'header': {'date': datetime.datetime(2024, 10, 8, 20, 28, 32, 220000, tzinfo=tzutc()), 'msg_id': 'e379e97f-1526-4a8d-bc7f-1e481452416c', 'msg_type': 'comm_msg', 'session': '28950468-58ca-494e-b47f-fbd55678ef50', 'username': '18153ef9-50e0-4d4b-ad84-32d9963a420e', 'version': '5.2'}, 'msg_id': 'e379e97f-1526-4a8d-bc7f-1e481452416c', 'msg_type': 'co